<a href="https://colab.research.google.com/github/EmranAljawarneh/NLPDisasterTweets/blob/main/nlp_with_disaster_tweets_using_lstm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing with Disaster Tweets

This notebbok tried to build a deep learning model that predicts which Tweets are about real disasters and which one’s aren’t. To build this model, we should prepared the data to be suitable for the model to give the best possible results.
In this notebook, we used the Long Short-Term Memory (LSTM) deep learning which considers type of Recurrent Neural Network (RNN) deep learning model.

## Import Libraries

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import string, nltk
import re
from nltk.tokenize import word_tokenize
punct = nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer, PorterStemmer
lemma=WordNetLemmatizer()
stemm = PorterStemmer()
from nltk.tokenize import WordPunctTokenizer
tokenize = WordPunctTokenizer()

from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [ ]:
# Read Dataset
read_data = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
print(read_data.head())

   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  


In this task we need just two features from the whole dataset; text and target. So, we will drop the remain features; id, keyword, and location.

In [ ]:
data = read_data.drop(['id', 'keyword', 'location'], axis=1)
data

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


We should check the null values and remove the duplicated values that exists in the data.

In [ ]:
data.isnull().sum()
data = data.drop_duplicates()

We need to split the text and the target features to deal with the text data in the preprocessing phase.

In [ ]:
label_data = data.drop(['text'], axis=1)
train_data = data['text']
print(train_data.shape)
print(label_data.shape)

(7521,)
(7521, 1)


# Clean Data
Here, we will the data and process it to be suitable for the model building.
The preprocess steps are as follows:
1. Convert the whole data into lower case.
2. Remove numbers, html tags, urls, and punctuations.
3. Tokenize the text using Word Tokenize function.
4. Remove all stop words from the data.
5. Retrieve the text into their base form using the stemming and lemmatization function.

In [ ]:
def lower(text):
  return str(text.lower())

def remove_numbers(text):
  return re.sub('\d+', '', text)

def remove_html_tags(text):
  return re.sub('\[.*?\]', '', text)

def remove_url(text):
  return re.sub('https?://\S+|www\.\S+', '', text)

def remove_punctuations(text):
  return re.sub('[%s]' % re.escape(string.punctuation),'',text)

def remove_stop_words(text):
  return ' '.join([word for word in text if word.lower() not in stop_words])

def lemmatize_data(text):
  text = ' '.join(stemm.stem(word) for word in text.split(' '))
  text = ' '.join(lemma.lemmatize(word) for word in text.split(' '))
  return text

def wordTokenize(text):
  text = tokenize.tokenize(text)
  return text

This cell will how we call the above functions.

In [ ]:
def preprocess(text):
  text = lower(text)
  text = remove_numbers(text)
  text = remove_html_tags(text)
  text = remove_url(text)
  text = remove_punctuations(text)
  text = wordTokenize(text)
  text = remove_stop_words(text)
  text = lemmatize_data(text)
  return text

train_data = train_data.apply(preprocess)
train_data

0                deed reason earthquak may allah forgiv u
1                    forest fire near la rong sask canada
2       resid ask shelter place notifi offic evacu she...
3             peopl receiv wildfir evacu order california
4       got sent photo rubi alaska smoke wildfir pour ...
                              ...                        
7604    worldnew fallen powerlin glink tram updat fire...
7605    flip side im walmart bomb everyon evacu stay t...
7606    suicid bomber kill saudi secur site mosqu reut...
7608       two giant crane hold bridg collaps nearbi home
7612    latest home raze northern california wildfir a...
Name: text, Length: 7521, dtype: object

# LSTM Model

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Tokenize the texts
max_features=3000
tokenizer=Tokenizer(num_words=max_features,split=' ')
tokenizer.fit_on_texts(train_data.values)     # convert text into numerical data
X = tokenizer.texts_to_sequences(train_data.values)
X = pad_sequences(X)      # make the whole input text data on the same size.

In [ ]:
tokenizer.sequences_to_texts([[ 713,  154,   56, 1434,   14]])

['interest god car amaz peopl']

# Split the Data
To split the data, we used the train_test_split function with train size 80%, test size 20%, and random state = 41

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, label_data, test_size = 0.2, random_state =41)

In [ ]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout
from keras.models import Model, Sequential

max_features = 30000
embed_dim = 32

lstm_model = Sequential()
lstm_model.add(Embedding(max_features, embed_dim, input_length = X_train.shape[1]))
#lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(units=60, input_shape=(X_train.shape[1], 1), activation='relu', return_sequences=False))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(loss = 'binary_crossentropy', optimizer='adam' , metrics = ['accuracy'])

2023-02-08 22:39:01.336770: I tensorflow/compiler/xla/service/service.cc:171] XLA service 0x564b119f2530 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-02-08 22:39:01.336844: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (0): Host, Default Version
2023-02-08 22:39:01.338702: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [ ]:
lstm_model.fit(X_train, y_train, epochs = 10, batch_size=32, validation_data=(X_test, y_test))

2023-02-08 22:39:01.723770: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
188/188 [==============================] - 9s 38ms/step - loss: 0.5807 - accuracy: 0.6845 - val_loss: 0.5254 - val_accuracy: 0.7993
Epoch 2/10
188/188 [==============================] - 6s 33ms/step - loss: 0.4251 - accuracy: 0.8210 - val_loss: 0.4423 - val_accuracy: 0.8093
Epoch 3/10
188/188 [==============================] - 7s 37ms/step - loss: 0.3432 - accuracy: 0.8527 - val_loss: 0.5523 - val_accuracy: 0.8093
Epoch 4/10
188/188 [==============================] - 6s 34ms/step - loss: 0.2998 - accuracy: 0.8740 - val_loss: 0.6423 - val_accuracy: 0.7967
Epoch 5/10
188/188 [==============================] - 6s 33ms/step - loss: 0.2730 - accuracy: 0.8840 - val_loss: 1.4305 - val_accuracy: 0.7927
Epoch 6/10
188/188 [==============================] - 6s 34ms/step - loss: 0.2409 - accuracy: 0.9003 - val_loss: 1.8309 - val_accuracy: 0.7814
Epoch 7/10
188/188 [==============================] - 7s 38ms/step - loss: 0.2194 - accuracy: 0.9046 - val_loss: 1.9260 - val_accuracy: 0.7834

In [ ]:
y_pred = lstm_model.predict(X_test).round()
# Final evaluation of the model
scores = lstm_model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 77.48%


In [ ]:
from sklearn import metrics
train_accuracy = round(metrics.accuracy_score(y_train, lstm_model.predict(X_train).round())*100)
train_accuracy

95

In [ ]:
accuracy = round(accuracy_score(y_test,y_pred),3)
precision = round(precision_score(y_test,y_pred,average='weighted'),3)
recall = round(recall_score(y_test,y_pred,average='weighted'),3)

print(f'Accuracy of the model: {np.round(accuracy*100,2)}%')
print(f'Precision Score of the model: {np.round(precision*100,2)}%')
print(f'Recall Score of the model: {np.round(recall*100,2)}%')
print('-'*50)
print(classification_report(y_test,y_pred))

Accuracy of the model: 77.5%
Precision Score of the model: 77.4%
Recall Score of the model: 77.5%
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.79      0.83      0.81       856
           1       0.76      0.70      0.73       649

    accuracy                           0.77      1505
   macro avg       0.77      0.77      0.77      1505
weighted avg       0.77      0.77      0.77      1505



# Evaluation the model
In this section, we will use the test file for model evaluation.

In [ ]:
test_data = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
test_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
data = test_data.drop(['keyword', 'location'], axis=1)
data.shape
id = data['id']
id = id.to_frame()
id = pd.DataFrame(id)
id

,id
0,0
1,2
2,3
3,9
4,11
...,...
3258,10861
3259,10865
3260,10868
3261,10874


In [ ]:
data.isnull().sum()
data = data.drop_duplicates()

In [ ]:
test_data = test_data['text'].apply(preprocess)
test_data

0                                happen terribl car crash
1           heard earthquak differ citi stay safe everyon
2       forest fire spot pond gee flee across street c...
3                          apocalyps light spokan wildfir
4                      typhoon soudelor kill china taiwan
                              ...                        
3258    earthquak safeti lo angel  ûò safeti fasten xrwn
3259    storm ri wors last hurrican cityampoth hardest...
3260                            green line derail chicago
3261                  meg issu hazard weather outlook hwo
3262      cityofcalgari activ municip emerg plan yycstorm
Name: text, Length: 3263, dtype: object

In [ ]:
test_token = tokenizer.texts_to_sequences(test_data.values)
test_token = pad_sequences(test_token, maxlen =1) # to make the whole text with the same size.

In [ ]:
final_pred = lstm_model.predict(test_token)

In [ ]:
final_pred = np.round(final_pred).astype(int)
final_pred

array([[1],
       [1],
       [0],
       ...,
       [1],
       [1],
       [0]])

In [ ]:
id = pd.DataFrame(id, columns=['id'])
final_pred = pd.DataFrame(final_pred, columns=['target'])
submission = pd.concat([id, final_pred], axis=1, join='inner')
submission

,id,target
0,0,1
1,2,1
2,3,0
3,9,1
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,1


In [ ]:
submission.to_csv('submission.csv', index=False)